In [1]:
import torch

# einsum
torch.einsum(equation, *operands)<br>
공식홈페이지 : Sums the product of the elements of the input operands along dimensions specified using a notation based on the Einstein summation convention.

numpy, tensorflow에도 동일한 연산이 있음

equation은 inputs->output이며 각 tensor의 shape을 나타냄, inputs는 ","를 통해 여러 개를 구분함

In [22]:
def check(equation,  *matrix, input_print=True):
    if input_print:
        print('Before operating')
        for mat in matrix:
            print(mat)
    einsum = torch.einsum(equation, *matrix)
    print('After operating')
    print(einsum)

In [23]:
# 1개의 tensor에 대한 sum
test = (torch.rand(2,3)*20).int()
check('ij->', test)
# 행의 개수를 유지한채 반환되도록 행별로 더함
check('ij->i', test, input_print=False)
# 열의 개수를 유지한채 반환되로독 열별로 더함
check('ij->j', test, input_print=False)

Before operating
tensor([[ 5, 13,  5],
        [ 6, 14, 15]], dtype=torch.int32)
After operating
tensor(58)
After operating
tensor([23, 35])
After operating
tensor([11, 27, 20])


In [34]:
# 여러개의 tensor에 대한 
test1 = torch.Tensor([[0,1,2],[3,4,5]])
test2 = torch.Tensor([0,1,2])
check('ij, j->ij', test1, test2, input_print=True)
check('ij, j->i', test1, test2, input_print=False)
check('ik,kj->ij', test1, test1.T, input_print=True)

Before operating
tensor([[0., 1., 2.],
        [3., 4., 5.]])
tensor([0., 1., 2.])
After operating
tensor([[ 0.,  1.,  4.],
        [ 0.,  4., 10.]])
After operating
tensor([ 5., 14.])
Before operating
tensor([[0., 1., 2.],
        [3., 4., 5.]])
tensor([[0., 3.],
        [1., 4.],
        [2., 5.]])
After operating
tensor([[ 5., 14.],
        [14., 50.]])


# repeat & repeat_interleave

- repeat은 어느 dimension으로 반복시킬지 지정

In [10]:
x = torch.Tensor([[1,2,3],[4,5,6]])
print(x)
print('\ndim=0으로 2번, dim=1로 3번\n',x.repeat(2,3))


tensor([[1., 2., 3.],
        [4., 5., 6.]])

dim=0으로 2번, dim=1로 3번
 tensor([[1., 2., 3., 1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6., 4., 5., 6.],
        [1., 2., 3., 1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6., 4., 5., 6.]])


- repeat_interleave는 element 하나하나를 반복시킴
- 차원별로 repeat의 횟수를 다르게 설정할 수 있음
- 어느 차원으로 repeat할지 설정 가능
- repeats 인자는 Tensor or int, dim 인자는 int

In [32]:
x = torch.Tensor([[1,2,3],[4,5,6]])
print(x)
print('\ndim=0으로 2번 반복 : \n' ,x.repeat_interleave(2, dim=0))
print('\ndim=1으로 2번 반복 : \n' ,x.repeat_interleave(2, dim=1))
print('\ndim=1으로 2번 반복 : \n' ,x.repeat_interleave(2, dim=1))
print('\ndim=0로 x의 행에 대해서는 2번, x의 열에 대해서는 3번 반복\n',x.repeat_interleave(repeats=torch.tensor([2,3]), dim=0))

tensor([[1., 2., 3.],
        [4., 5., 6.]])

dim=0으로 2번 반복 : 
 tensor([[1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.]])

dim=1으로 2번 반복 : 
 tensor([[1., 1., 2., 2., 3., 3.],
        [4., 4., 5., 5., 6., 6.]])

dim=1으로 2번 반복 : 
 tensor([[1., 1., 2., 2., 3., 3.],
        [4., 4., 5., 5., 6., 6.]])

dim=0로 x의 행에 대해서는 2번, x의 열에 대해서는 3번 반복
 tensor([[1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])


# Convolution filter
filter는 output개수 * 3차원 모양<br>
각 channel에 대해 다음과 같이 연산됨

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

temp = torch.randn(3,32,32)
conv = nn.Conv2d(3, 6, kernel_size=(4,4), bias=False)
temp2 = conv(temp)
print(f'input shape : {temp.shape}, output shape : {temp2.shape}, filter shape : {conv.weight.shape}')

input shape : torch.Size([3, 32, 32]), output shape : torch.Size([6, 29, 29]), filter shape : torch.Size([6, 3, 4, 4])


In [4]:
a,b,c = conv.weight[0,0], conv.weight[0,1], conv.weight[0,2]
aa = F.conv2d(temp[0].unsqueeze(0), a.unsqueeze(0).unsqueeze(0))
bb = F.conv2d(temp[1].unsqueeze(0), b.unsqueeze(0).unsqueeze(0))
cc = F.conv2d(temp[2].unsqueeze(0), c.unsqueeze(0).unsqueeze(0))

print((aa+bb+cc)[0,:3,:3])
print(temp2[0,:3,:3])

tensor([[ 0.7445, -1.3043, -0.7599],
        [-0.0114,  1.1250, -0.2401],
        [ 0.2970,  0.9768,  0.1136]], grad_fn=<SliceBackward0>)
tensor([[ 0.7445, -1.3043, -0.7599],
        [-0.0114,  1.1250, -0.2401],
        [ 0.2970,  0.9768,  0.1136]], grad_fn=<SliceBackward0>)
